# 09 - Scenario Generator

## Agentic Logistics Control System

This notebook implements scenario generation for the PLANNING phase:
- Generate alternatives for issues
- Evaluate and compare scenarios
- Complete planning node integration

In [ ]:
import sys
from pathlib import Path
import uuid
from datetime import datetime

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from src.models import *
from src.planning.scenario_generator import ScenarioGenerator
from src.planning.simulation_engine import SimulationEngine
from src.planning.planning_node import create_planning_node
from src.perception.observation_node import create_sample_fleet

print("Scenario generator loaded!")

In [ ]:
# Create test issue
issue = Issue(
    id="ISSUE-001",
    type="stuck",
    severity="high",
    description="Truck TRK-001 stuck due to traffic",
    affected_truck_ids=["TRK-001"],
    affected_load_ids=["LOAD-001"],
)

fleet = create_sample_fleet()
fleet[0].status = TruckStatus.STUCK

loads = [Load(
    id="LOAD-001", description="Urgent delivery", weight_kg=1000,
    priority=LoadPriority.URGENT,
    pickup_location=Location(latitude=40.71, longitude=-74.00),
    delivery_location=Location(latitude=40.75, longitude=-73.98),
    assigned_truck_id="TRK-001"
)]

traffic = [TrafficCondition(segment_id="SEG-1", level=TrafficLevel.STANDSTILL, speed_kmh=5)]

In [ ]:
# Generate scenarios
generator = ScenarioGenerator()
scenarios = generator.generate_scenarios(issue, fleet, loads, traffic)

print(f"Generated {len(scenarios)} scenarios:")
for s in scenarios:
    print(f"\n  {s.name}")
    print(f"    Time: {s.estimated_time_minutes:.0f} min")
    print(f"    Cost: ${s.estimated_cost:.2f}")
    print(f"    Reliability: {s.reliability_score:.2f}")

In [ ]:
# Compare scenarios
engine = SimulationEngine()
comparison = engine.compare_scenarios(scenarios)

print("\nScenario Comparison:")
for sid, scores in comparison.items():
    print(f"  {sid}: overall={scores['overall']:.3f}")

In [ ]:
# Test planning node
from src.perception.observation_node import create_observation_node
from src.reasoning.reasoning_node import create_reasoning_node

# Create initial state
initial_state: AgentState = {
    "current_phase": ControlLoopPhase.OBSERVE,
    "cycle_id": str(uuid.uuid4()),
    "trucks": [t.model_dump() for t in fleet],
    "routes": [], "loads": [], "traffic_conditions": [],
    "gps_readings": [], "observation_timestamp": "",
    "reasoning_result": None, "current_issues": [],
    "planning_result": None, "scenarios": [],
    "decision_result": None, "selected_decision": None,
    "action_results": [], "notifications_sent": [],
    "feedback_result": None, "continue_loop": True,
    "requires_human_intervention": False, "error_message": None,
    "cycle_start_time": datetime.utcnow().isoformat(),
    "cycle_end_time": None, "total_cycles": 0,
}

# Run OBSERVE -> REASON -> PLAN
observe = create_observation_node()
reason = create_reasoning_node()
plan = create_planning_node()

state = observe(initial_state)
state = reason(state)
state = plan(state)

print(f"Planning completed:")
print(f"  Scenarios: {len(state['scenarios'])}")
print(f"  Recommended: {state['planning_result'].get('recommended_scenario_id')}")

## Phase 3 Complete! Next: Phase 4 Decision Layer